In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json

In [2]:
CLIENT_ID = 'C2J0PSOHKIRG5ESKPR5K32K04ZHYRFJ45L2IMSAC205NSHHQ' # your Foursquare ID
CLIENT_SECRET = 'GIILSQVIVJIGT12PYDIATN1CY5B2IACB5NUM2WZGUIJT5LTN' # your Foursquare Secret
VERSION = '20180604'
limit=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C2J0PSOHKIRG5ESKPR5K32K04ZHYRFJ45L2IMSAC205NSHHQ
CLIENT_SECRET:GIILSQVIVJIGT12PYDIATN1CY5B2IACB5NUM2WZGUIJT5LTN


# Get Coordinates of different neighborhoods of Madrid

In [3]:

coordinates_df = pd.DataFrame( data = {

'Neighborhood' : ['Centro', 'Arganzuela', 'Retiro', 'Salamanca', 'Chamartin',
                'Tetuán', 'Chamberí', 'Fuencarral-El Pardo', 'Moncloa-Aravaca', 'Latina', 'Carabanchel',
                'Usera', 'Puente de Vallecas', 'Moratalaz', 'Ciudad Lineal', 'Hortaleza', 'Villaverde',
                'Villa de Vallecas', 'Vicálvaro', 'San Blas-Canillejas', 'Barajas'],
'Latitude' : ['40.415347', '40.402733', '40.408072', '40.43', '40.453333', '40.460556', '40.432792', '40.478611', '40.435151',
                 '40.402461', '40.383669', '40.381336', '40.398204', '40.409869', '40.45', '40.469457', '40.345925', '40.3796', 
                 '40.4042', '40.426001', '40.470196'],
'Longitude' : ['-3.707371', '-3.695403', '-3.676729', '-3.677778', '-3.6775', '-3.7', '-3.697186', '-3.709722', '-3.718765', 
                  '-3.741294', '-3.727989', '-3.706856', '-3.669059', '-3.644436', '-3.65', '-3.640482', '-3.709356', '-3.62135', 
                  '-3.60806', '-3.612764', '-3.58489']})

coordinates_df_sorted = coordinates_df.sort_values('Neighborhood').reset_index()
coordinates_df_sorted.head()

,index,Neighborhood,Latitude,Longitude
0,1,Arganzuela,40.402733,-3.695403
1,20,Barajas,40.470196,-3.58489
2,10,Carabanchel,40.383669,-3.727989
3,0,Centro,40.415347,-3.707371
4,4,Chamartin,40.453333,-3.6775


# Function to call Foursquare API

In [4]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



In [5]:
LIMIT=100
Madrid_venues = getNearbyVenues(names= coordinates_df['Neighborhood'],
                                   latitudes= coordinates_df['Latitude'],
                                   longitudes= coordinates_df['Longitude']
                                  )

Centro
Arganzuela
Retiro
Salamanca
Chamartin
Tetuán
Chamberí
Fuencarral-El Pardo
Moncloa-Aravaca
Latina
Carabanchel
Usera
Puente de Vallecas
Moratalaz
Ciudad Lineal
Hortaleza
Villaverde
Villa de Vallecas
Vicálvaro
San Blas-Canillejas
Barajas


In [6]:
Madrid_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Centro,40.415347,-3.707371,Plaza Mayor,40.415527,-3.707506,Plaza
1,Centro,40.415347,-3.707371,Amorino,40.416065,-3.708383,Ice Cream Shop
2,Centro,40.415347,-3.707371,La Taberna de Mister Pinkleton,40.414536,-3.708108,Other Nightlife
3,Centro,40.415347,-3.707371,Mercado de San Miguel,40.415443,-3.708943,Market
4,Centro,40.415347,-3.707371,The Hat Madrid,40.414343,-3.707120,Hotel


# Plot all venues retrieved in map

In [7]:
from folium import plugins
import folium
#madrid coordinates

madrid_geo=r'geojson/distritos-1.geojson'
latitude_mad = 40.4167754
longitude_mad = -3.7037902
# let's start again with a clean copy of the map of San Francisco
madrid_map = folium.Map(location = [latitude_mad, longitude_mad], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(madrid_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(Madrid_venues['Venue Latitude'], Madrid_venues['Venue Longitude'], Madrid_venues['Venue Category']):
    folium.Marker(
        geo_data=madrid_geo,
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# display map
madrid_map

# We are going to segment the venues in different clusters attending to the distance between them. The objective of this segmentation is to group the venues that are close together, in walking distance from each other

In [8]:
from sklearn.cluster import KMeans
X1=[Madrid_venues['Venue Latitude'], Madrid_venues['Venue Longitude']]
X = np.array(X1).T.tolist()

## The number of neighborhoods in Madrid is 21. This is why we have to choose a number of clusters higher than that. If we choose 21 or lower, we have the issue of segmenting into the existing neighborhoods
## However, when we choose a higher number, it can further divide the venues inside a neighborhood into walking distance 

In [9]:

kclusters=30
k_means = KMeans(init = "k-means++", n_clusters = kclusters, n_init = 12)
k_means.fit(X)
k_means_labels = k_means.labels_
Madrid_venues['labels']=k_means_labels
Madrid_venues.dtypes

Neighborhood               object
Neighborhood Latitude      object
Neighborhood Longitude     object
Venue                      object
Venue Latitude            float64
Venue Longitude           float64
Venue Category             object
labels                      int32
dtype: object

In [10]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude_mad, longitude_mad], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(Madrid_venues['Venue Latitude'],Madrid_venues['Venue Longitude'],Madrid_venues['labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# We are going to analyze the income of different neighborhoods in Madrid and choose the top 5 neighborhoods

In [11]:
mean_income=pd.read_csv('buscador_de_renta_media_p.csv',sep=';')
mean_income.head()

,Año,Periodo,Parámetro,Renta bruta media,Renta disponible media
0,2018,Año,02001-San Antonio Abad- La Pajarita- Polígono ...,32.036,25.823
1,2018,Año,02002-Parque Sur-Huerta de Marzo (Albacete),30.426,24.615
2,2018,Año,02003-Franciscanos (Albacete),26.866,22.145
3,2018,Año,02004-Fátima- Villacerrada (Albacete),23.072,19.449
4,2018,Año,02005-Industria- El Pilar- Feria- Ntra. Sra. d...,25.098,20.822


In [12]:
madrid_income1=mean_income[mean_income['Parámetro'].str.contains('Madrid',na=False)]


madrid_income =madrid_income1.sort_values('Renta disponible media',ascending=False)
madrid_income.columns=['Year','Period','Place','Mean income','Available income']
madrid_income.head(10)

,Year,Period,Place,Mean income,Available income
230,2018,Año,28001-Salamanca- Goya (Madrid),95.492,71.599
252,2018,Año,28023-Aravaca (Madrid),95.427,69.536
235,2018,Año,28006-Castellana (Madrid),91.283,68.569
265,2018,Año,28036-Nueva España (Madrid),86.202,64.138
239,2018,Año,28010-Almagro (Madrid),72.484,54.920
275,2018,Año,28046-Castilla -Chamartín (Madrid),72.809,54.526
245,2018,Año,28016-Hispanoamerica-Costillares (Madrid),68.463,51.338
283,2018,Año,28055-Valdebebas-Valdefuentes (Madrid),64.753,47.929
243,2018,Año,28014-Retiro (Madrid),61.845,47.443
237,2018,Año,28008-Moncloa (Madrid),56.732,43.807


### We are going to choose only the places with available income > 50.000

In [13]:
madrid_top = madrid_income.head(7)
madrid_top

,Year,Period,Place,Mean income,Available income
230,2018,Año,28001-Salamanca- Goya (Madrid),95.492,71.599
252,2018,Año,28023-Aravaca (Madrid),95.427,69.536
235,2018,Año,28006-Castellana (Madrid),91.283,68.569
265,2018,Año,28036-Nueva España (Madrid),86.202,64.138
239,2018,Año,28010-Almagro (Madrid),72.484,54.920
275,2018,Año,28046-Castilla -Chamartín (Madrid),72.809,54.526
245,2018,Año,28016-Hispanoamerica-Costillares (Madrid),68.463,51.338


### Added a column to shown which neighborhood does the place belongs to

In [14]:
place=['Salamanca','Moncloa-Aravaca','Chamartin','Chamartin','Chamberi','Chamartin','Chamartin']
madrid_top['Neighborhood']=place

<ipython-input-14-0c4137cb8a24>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  madrid_top['Neighborhood']=place


In [15]:
madrid_top

,Year,Period,Place,Mean income,Available income,Neighborhood
230,2018,Año,28001-Salamanca- Goya (Madrid),95.492,71.599,Salamanca
252,2018,Año,28023-Aravaca (Madrid),95.427,69.536,Moncloa-Aravaca
235,2018,Año,28006-Castellana (Madrid),91.283,68.569,Chamartin
265,2018,Año,28036-Nueva España (Madrid),86.202,64.138,Chamartin
239,2018,Año,28010-Almagro (Madrid),72.484,54.920,Chamberi
275,2018,Año,28046-Castilla -Chamartín (Madrid),72.809,54.526,Chamartin
245,2018,Año,28016-Hispanoamerica-Costillares (Madrid),68.463,51.338,Chamartin


### As we can see here, the neighborhoods with most income with the parameters we have set are Salamanca, Moncloa-Aravaca, Chamartín and Chamberí. Which are all in the northern part of Madrid. These neighnorhoods are perfect for openning a new restaurant.
### The clusters are 1-28-19 for Chamberí, 6 for Moncloa-Aravaca, 13-25 for Salamanca and 8 for Chamartin

#### Split clusters in different DF

In [16]:
grouped = Madrid_venues.groupby(Madrid_venues.labels)
clusterss=['Cluster 1','Cluster 28','Cluster 19','Cluster 6','Cluster 13','Cluster 25','Cluster 8']
cluster1 = grouped.get_group(1)
cluster28 = grouped.get_group(28)
cluster19 = grouped.get_group(19)

cluster6 = grouped.get_group(6)

cluster13 = grouped.get_group(13)
cluster25 = grouped.get_group(25)

cluster8 = grouped.get_group(8)


total=[cluster1['Venue Category'],cluster28['Venue Category'],cluster19['Venue Category'],cluster6['Venue Category'],cluster13['Venue Category'],cluster25['Venue Category'],cluster8['Venue Category']]
total1=pd.DataFrame(total)

#total1['{}'.format('Cluster 1')].value_counts()
total1.index=clusterss
total1=total1.T


#### Get frequency of types of venues

In [35]:

total1['Cluster 25'].value_counts()[:n].index.tolist()
total1['Cluster 25'].value_counts()

Plaza                   3
Ice Cream Shop          2
Opera House             1
Gym / Fitness Center    1
Japanese Restaurant     1
Beer Bar                1
Dumpling Restaurant     1
Ramen Restaurant        1
Bistro                  1
Chocolate Shop          1
Pastry Shop             1
Smoothie Shop           1
Historic Site           1
Spanish Restaurant      1
Hostel                  1
Market                  1
Bookstore               1
Restaurant              1
Name: Cluster 25, dtype: int64

### Calculate the 10 most frequent venues per cluster

In [32]:

n = 10


data=[]
for i,cluster in zip(range(total1.shape[1]),clusterss):
    a=total1['{}'.format(cluster)].value_counts()[:n].index.tolist()
    data.append(a)


clusters_sorted=pd.DataFrame(data)


import numpy as np


indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = []
for ind in np.arange(n):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))



clusters_sorted.columns=columns
clusters_sorted.index=clusterss

In [33]:
clusters_sorted

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster 1,Spanish Restaurant,Supermarket,Argentinian Restaurant,Gastropub,Restaurant,Burger Joint,Bakery,Music Venue,Ice Cream Shop,Pizza Place
Cluster 28,Restaurant,Spanish Restaurant,Grocery Store,Bakery,Sandwich Place,Hotel,Electronics Store,Snack Place,Tattoo Parlor,Gym
Cluster 19,Train Station,Food,Metro Station,Plaza,Grocery Store,Soccer Field,Spanish Restaurant,Park,None,None
Cluster 6,Pizza Place,Spanish Restaurant,Asian Restaurant,Restaurant,Camera Store,Grocery Store,Beer Bar,Breakfast Spot,Sandwich Place,Fast Food Restaurant
Cluster 13,Bakery,Art Studio,Brazilian Restaurant,Supermarket,Music Venue,Clothing Store,Spanish Restaurant,Sandwich Place,Restaurant,None
Cluster 25,Plaza,Ice Cream Shop,Opera House,Gym / Fitness Center,Japanese Restaurant,Beer Bar,Dumpling Restaurant,Ramen Restaurant,Bistro,Chocolate Shop
Cluster 8,Seafood Restaurant,Theater,Plaza,Chinese Restaurant,Bubble Tea Shop,Noodle House,Spanish Restaurant,Fast Food Restaurant,Asian Restaurant,None


### The first 3 clusters, which correspond to the neighborhood of Chamberi, have as most common venues some sort of restaurants (eg. Spanish restaurant, restaurant, Argentinian restaurant...). this means that this place is a very popular place for eating out. The available income of this part of Madrid is around 54920€ in 2018 which suggests that this is indeed a wealthy part so the positioning of the new restaurant can be luxurious. 

### Cluster 6, corresponds to Moncloa-Aravaca, which is where universities are located. And as expected, pizza places are the most popular due to the nature of the place and university students. This is followed by Spanish restaurants, which can be "Bar de tapas" which confirms the location of universities

### Chamartin has the highest amount of seafood restaurants and also Asian styled venues such as Bubble Tea Shop, Chinese Restaurant, an Asian restaurant. 

### Lastly, clusters 13 and 25 are located in Salamanca, one of the highest-earning neighborhoods in Madrid. There is a variety of venues in this section and not dominated by restaurants as we can see in other sectors. This area is perfect to start a new business due to its high income and relative lack of restaurants. 

### The perfect restaurant for the area of Salamanca would be a seafood restaurant due to the high income of this neighborhood and the lack of options available.



Disclaimer: These data are extracted from Foursquare API which has a limit to the number of venues you can retrieve. Meaning that it does not offer a complete view of venues around the area.


